##### Copyright 2024 Google LLC.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini 2.0 - Multimodal live API: Tool use

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/gemini-2/live_api_tool_use.ipynb"><img src="https://github.com/google-gemini/cookbook/blob/main/images/colab_logo_32px.png?raw=1" />Run in Google Colab</a>
  </td>
</table>

This notebook provides examples of how to use tools with the multimodal live API with [Gemini 2.0](https://ai.google.dev/gemini-api/docs/models/gemini-v2).

The API provides Google Search, Code Execution and Function Calling tools. The earlier Gemini models supported versions of these tools. The biggest change with Gemini 2 (in the Live API) is that, basically, all the tools are handled by Code Execution. With that change, you can use **multiple tools** in a single API call, and the model can use multiple tools in a single code execution block.  

This tutorial assumes you are familiar with the Live API, as described in the [this tutorial](https://github.com/google-gemini/cookbook/blob/main/gemini-2/live_api_starter.ipynb).

## Setup

### Install SDK

The new **[Google Gen AI SDK](https://ai.google.dev/gemini-api/docs/sdks)** provides programmatic access to Gemini 2.0 (and previous models) using both the [Google AI for Developers](https://ai.google.dev/gemini-api/docs) and [Vertex AI](https://cloud.google.com/vertex-ai/generative-ai/docs/overview) APIs. With a few exceptions, code that runs on one platform will run on both. This means that you can prototype an application using the Developer API and then migrate the application to Vertex AI without rewriting your code.

More details about this new SDK on the [documentation](https://ai.google.dev/gemini-api/docs/sdks) or in the [Getting started](../gemini-2/get_started.ipynb) notebook.

In [ ]:
!pip install -U -q google-genai

### Setup your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see [Authentication](../quickstarts/Authentication.ipynb) for an example.

In [ ]:
from google.colab import userdata
import os

os.environ['GOOGLE_API_KEY']=userdata.get('GOOGLE_API_KEY')

### Initialize SDK client

The client will pickup your API key from the environment variable.
To use the live API you need to set the client version to `v1alpha`.

In [ ]:
from google import genai

client = genai.Client(http_options= {
      'api_version': 'v1alpha'
})

### Select a model

Multimodal Live API are a new capability introduced with the [Gemini 2.0](https://ai.google.dev/gemini-api/docs/models/gemini-v2) model. It won't work with previous generation models.

In [ ]:
model_name = "gemini-2.0-flash-exp"

### Imports

In [43]:
import asyncio
import contextlib
import json
import wave
import random

from IPython import display

from google import genai
from google.genai import types

### Utilities

You're going to use the Live API's audio output, the easiest way hear it in Colab is to write the `PCM` data out as a `WAV` file:

In [ ]:
@contextlib.contextmanager
def wave_file(filename, channels=1, rate=24000, sample_width=2):
    with wave.open(filename, "wb") as wf:
        wf.setnchannels(channels)
        wf.setsampwidth(sample_width)
        wf.setframerate(rate)
        yield wf

Use a logger so it's easier to switch on/off debugging messages.

In [ ]:
import logging
logger = logging.getLogger('Live')
#logger.setLevel('DEBUG')  # Switch between "INFO" and "DEBUG" to toggle debug messages.
logger.setLevel('INFO')

## Get started

Most of the Live API setup will be similar to the [starter tutorial](../gemini-2/live_api_starter.ipynb). Since this tutorial doesn't focus on the realtime interactivity of the API, the code has been simplified: This code uses the Live API, but it only sends a single text prompt, and listens for a single turn of replies.

You can set `modality="AUDIO"` on any of the examples to get the spoken version of the output.

In [18]:
n = 0
async def run(prompt, modality="TEXT", tools=None):
  global n
  if tools is None:
    tools=[]

  config = {
          "tools": tools,
          "generation_config": {
              "response_modalities": [modality]}}

  async with client.aio.live.connect(model=model_name, config=config) as session:
    display.display(display.Markdown(prompt))
    display.display(display.Markdown('-------------------------------'))
    await session.send(input=prompt, end_of_turn=True)

    audio = False
    filename = f'audio_{n}.wav'
    with wave_file(filename) as wf:
      async for response in session.receive():
        logger.debug(str(response))
        if text:=response.text:
          display.display(display.Markdown(text))
          continue

        if data:=response.data:
          print('.', end='')
          wf.writeframes(data)
          audio = True
          continue

        server_content = response.server_content
        if server_content is not None:
          handle_server_content(wf, server_content)
          continue

        tool_call = response.tool_call
        if tool_call is not None:
          await handle_tool_call(session, tool_call)


  if audio:
    display.display(display.Audio(filename, autoplay=True))
    n = n+1

Since this tutorial demonstrates several tools, you'll need more code to handle the different types of objects it returns.

- The `code_execution` tool can return `executable_code` and `code_execution_result` parts.
- The `google_search` tool may attach a `grounding_metadata` object.

In [19]:
def handle_server_content(wf, server_content):
  model_turn = server_content.model_turn
  if model_turn:
    for part in model_turn.parts:
      executable_code = part.executable_code
      if executable_code is not None:
        display.display(display.Markdown('-------------------------------'))
        display.display(display.Markdown(f'``` python\n{executable_code.code}\n```'))
        display.display(display.Markdown('-------------------------------'))

      code_execution_result = part.code_execution_result
      if code_execution_result is not None:
        display.display(display.Markdown('-------------------------------'))
        display.display(display.Markdown(f'```\n{code_execution_result.output}\n```'))
        display.display(display.Markdown('-------------------------------'))

  grounding_metadata = getattr(server_content, 'grounding_metadata', None)
  if grounding_metadata is not None:
    display.display(
        display.HTML(grounding_metadata.search_entry_point.rendered_content))

  return

- Finally, with the `function_declarations` tool, the API may return `tool_call` objects. To keep this code minimal, the `tool_call` handler just replies to every function call with a response of `"ok"`.

In [47]:

def fn_generate_kyc_profile():
  risk_level = random.choice(["low", "medium", "high"])
  response = client.models.generate_content(
    model=model_name,
    contents=f"""
    You are an expert in Anti Money Laundering,
    and you are helping a product team to build a simulation tool for KYC analysis.
    Please generate a typical {risk_level} client profile and data which is used for KYC analysis"""
  )
  return response.text


FUN_TOOLS = {"analyze_kyc_profile": fn_analyze_kyc_profile, "generate_kyc_profile": fn_generate_kyc_profile}

In [48]:
async def handle_tool_call(session, tool_call):
  for fc in tool_call.function_calls:

    res = FUN_TOOLS[fc.name](**fc.args)
    print(fc)
    print(res)
    tool_response = types.LiveClientToolResponse(
        function_responses=[types.FunctionResponse(
            name=fc.name,
            id=fc.id,
            response={'result':res},
        )]
    )

    print('\n>>> ', tool_response)
    await session.send(input=tool_response)

Try running it for a first time:

In [45]:
await run(prompt="Can you explain what is Non-Financial risk for a bank? Please be concise", tools=None, modality = "AUDIO")

Can you explain what is Non-Financial risk for a bank? Please be concise

-------------------------------

...................................................................

## Simple function call

The function calling feature of the API Can handle a wide variety of functions. Support in the SDK is still under construction. So keep this simple just send a minimal function definition: Just the function's name.

Note that in the live API function calls are independent of the chat turns. The conversation can continue while a function call is being processed.

In [57]:
generate_kyc_profile = types.FunctionDeclaration(
    name="generate_kyc_profile",
    description="Simulate the KYC profile of a fictive client",
)

/usr/local/lib/python3.11/dist-packages/google/genai/_common.py:220: UserWarning: JSON is not a valid Type
  warnings.warn(f"{value} is not a valid {cls.__name__}")


In [58]:
prompt = """Simulate an typical anti-money laundering analyze using simulated customer KYC profiles,
 and the standard KYC analyse methods. Please keep it short"""

tools = [
    {'function_declarations': [generate_kyc_profile]}
]

await run(prompt, tools=tools, modality = "TEXT")

Simulate an typical anti-money laundering analyze using simulated customer KYC profiles,
 and the standard KYC analyse methods

-------------------------------

I

 understand that you're asking to simulate an anti-money laundering (AML)

 analysis using simulated customer KYC profiles and standard KYC analysis methods.

The `generate

...



**Important Considerations:**

*   **Data Limitations

:** The `generate_kyc_profile` function provides limited data. For a realistic simulation, you would need access to more comprehensive customer information.
*   **Complexity:** Real AML analysis is much more complex, involving sophisticated algorithms, machine learning models, and large datasets of historical transactions.
*  **Regulatory

 Compliance:**  AML regulations vary between jurisdictions, and it is crucial to adapt your analysis methods based on the specific legal requirements.
*   **This is a simplified illustration.** Actual AML analysis requires expertise, specialized tools, and knowledge of specific regulations.

This example will give you a basic framework to begin your simulation.

  You can enhance it by:

*   Adding more complex risk scoring criteria.
*   Defining a broader range of red flags.
*   Integrating external data sources (e.g., sanctions lists).
*   Implementing machine learning for risk assessment.

Remember to always consult with AML experts and ensure compliance

 with relevant regulations when developing real AML solutions.


## Code execution

The `code_execution` lets the model write and run python code. Try it on a math problem the model can't solve from memory:

In [59]:
prompt="Can you compute the largest prime palindrome under 100000."

tools = [
    {'code_execution': {}}
]

await run(prompt, tools=tools, modality="TEXT")

Can you compute the largest prime palindrome under 100000.

-------------------------------

Okay

, I can definitely do that! Here's my plan:

1.

 **Generate Palindromes:** I'll create a function to generate palindrom

...

```
98689

```

-------------------------------

Okay

, the code executed and found the largest prime palindrome under 1000

00 to be 98689.


## Compositional Function Calling

Compositional function calling refers to the ability to combine user defined functions with the `code_execution` tool. The model will write them into larger blocks of code, and then pause execution while it waits for you to send back responses for each call.


In [60]:
prompt="""You are an expert in AML. Please demostrate a typical Anti-Money Laundering analyze using simulated customer KYC profiles,
 and the standard KYC analyse methods. Please keep it short."""

tools = [
    {'code_execution': {}},
    {'function_declarations': [generate_kyc_profile]}
]

await run(prompt, tools=tools, modality="TEXT")

Simulate an typical anti-money laundering analyze using simulated customer KYC profiles,
 and the standard KYC analyse methods

-------------------------------

Okay

, I understand. I'll simulate a KYC profile using the provided API and

 then perform a basic analysis mimicking AML checks. This will involve looking for potential red

...

.
2.  **Ongoing Monitoring:** Implement the following:
    *   Continuous transaction monitoring to identify any unusual activity.
    *   Regular PEP and sanctions screenings
    *   Regular screening against adverse media databases.
    *   Periodic review of the KYC profile
3.  **Documentation**: Focus

 on getting the documents validated by a third party. This also needs to be an ongoing process.

**Further Development:**

*   **Risk Weighting:** The risk factors and their weights can be further refined, based on expert judgement and statistical analysis.  A more complex model will consider the interaction of different factors.


*   **Thresholds:** Define thresholds that trigger alerts. For example, a risk score above 0.7 may flag the need for immediate review.
*   **Scenario Testing:**  Run simulations with different scenarios (as outlined by the API response), to see how changes in the data impact the risk score

 and flag potential red flags.

This exercise demonstrates a basic AML analysis by applying standard KYC principles, identifying risk factors, creating a risk scoring approach and suggesting appropriate actions. The risk score can be used as a trigger for further action and as a tool to prioritize cases that require deeper analysis. It is also important that this

 should be an ongoing process.


## Google search

The `google_search` tool lets the model conduct google searches. For example, try asking it about events that are too recent to be in the training data.

The search will still execute in `AUDIO` mode, but you won't see the detailed results:

In [62]:
prompt="Can you use google search tell me about the most notorious Anti Money Laundary case in history?"

tools = [
   {'google_search': {}}
]

await run(prompt, tools=tools, modality="TEXT")

Can you use google search tell me about the most notorious Anti Money Laundary case in history?

-------------------------------

-------------------------------

``` python
print(google_search.search(queries=["most notorious anti-money laundering case in history", "largest money laundering case history"]))

```

-------------------------------

...

 in drug money.

**4. Bank of Credit and Commerce International (BCCI):** In the early 1980s, BCCI was one of the world's largest banks. However, investigations uncovered a massive web of crimes, including money laundering, fraud, arms trading, and prostitution. The bank

 was shut down and approximately $20 billion in value was lost.

**5. 1MDB Scandal:** This scandal involved the misappropriation of billions of dollars from a Malaysian government fund, 1Malaysia Development Berhad. High-ranking officials, including former Prime Minister Najib Razak, were accused of using

 the fund for personal gain and lavish lifestyles, involving complex financial transactions and money laundering schemes across multiple countries.

**Other notable cases include:**

*   **Al Capone:** The infamous Chicago gangster is often credited with popularizing the term "money laundering" by using laundromats and other businesses to disguise his illicit income

 from bootlegging and other criminal activities.
*   **The Bangladesh Bank Heist:** This cyber heist involved hackers infiltrating the Bangladesh Bank and attempting to transfer funds to accounts in other countries.
*   **Liberty Reserve:** This online payment processor was used to launder money, requiring minimal user registration, and facilitated

 transactions for various illicit activities.

These cases highlight the sophistication and wide-reaching impact of money laundering and the importance of robust anti-money laundering regulations and practices.


## Multi-tool


The biggest difference with the new API however is that you're no longer limited to using 1-tool per request. Try combining those tasks from the previous sections:

In [63]:
prompt = """You are an expert in AML.
Please demostrate a typical Anti-Money Laundering analyze using simulated customer KYC profiles,
and the standard KYC analyse methods.
Please use google search to ground your analysis and keep it short. Thanks!"""

tools = [
    {'google_search': {}},
    {'code_execution': {}},
    {'function_declarations': [generate_kyc_profile]}
]

await run(prompt, tools=tools, modality="TEXT")

You are an expert in AML. 
Please demostrate a typical Anti-Money Laundering analyze using simulated customer KYC profiles,
and the standard KYC analyse methods. 
Please use google search to ground your analysis and keep it short. Thanks!

-------------------------------

Okay

, I will demonstrate a typical AML analysis using simulated customer KYC profiles and standard KYC

 analysis methods. I'll also use Google Search to ground my analysis and keep

...

 scoring, and that there is a move toward information sharing.

**Summary**

This analysis demonstrates a typical AML process:

1.  **Initial KYC

/CDD:** A client profile is created and assessed.
2.  **Risk Assessment:** A risk level (low, medium, high) is assigned based on the client's profile.
3.  **Ongoing Monitoring:**

 Transactional behavior is continuously monitored for suspicious activity.
4.  **Enhanced Due Diligence (EDD):** EDD is triggered if red flags are detected or client risk goes up.
5.  **Suspicious Activity Reporting

:** Reports are filed with the appropriate authorities if necessary.

Jane Doe's simulated profile is assessed as low risk due to her stable profession, legitimate source of funds, low-risk location, and the absence of any red flags. However, ongoing monitoring and periodic reviews are essential to identify any potential changes in risk.



This is a simplified version, as the actual AML process is complex and involves a multitude of risk factors and different regulatory frameworks. However, this analysis provides a good overview.


## Next Steps

- For more information about the SDK see the [SDK docs](https://googleapis.github.io/python-genai/)
- This tutorial uses the high level SDK, if you're interested in the lower-level details, try the [Websocket version of this tutorial](../gemini-2/websocket/search_tool.ipynb)
- This tutorial only covers _basic_ usage of these tools for deeper (and more fun) example see the [Search tool tutorial](../gemini-2/search_tool.ipynb)

Or check the other Gemini 2.0 capabilities from the [Cookbook](https://github.com/google-gemini/cookbook/blob/main/gemini-2/), in particular this other [multi-tool](../gemini-2/plotting_and_mapping.ipynb) example and the one about Gemini [spatial capabilities](../gemini-2/spatial_understanding.ipynb).